In [ ]:
# !wget -O test.mcool https://osf.io/3h9js/download	

In [ ]:
import higlass as hg
import ipywidgets
import traitlets
import itertools

# TODO: Need to find an abstraction that brings this into notebooks.
class Tilesets(ipywidgets.DOMWidget):
    value = traitlets.Int(0).tag(sync=True)
    def __init__(self):
        super().__init__()
        self.on_msg(self._handle_custom_msg)
        self.ts = dict()

    def add(self, ts):
        self.ts[ts.tileset.uid] = ts.tileset
        return self

    def _handle_custom_msg(self, data, buffers):
        payload = data["payload"]
        uuid = data["uuid"]
        match payload:
            case { "type": "tileset_info", "tilesetUid": uid }:
                info = { uid: self.ts[uid].info() }
                self.send({ "uuid": uuid, "payload": info })
            case { "type": "tiles", "tileIds": tids }:
                all_tiles = []
                for uid, tids in itertools.groupby(
                    iterable=sorted(tids), key=lambda tid: tid.split(".")[0]
                ):
                    tiles = self.ts[uid].tiles(list(tids))
                    all_tiles.extend(tiles)
                data = {tid: tval for tid, tval in all_tiles}
                self.send({ "uuid": uuid, "payload": data })
            case _:
                raise ValueError("Something's wrong with the Internet")

ts = hg.cooler("./test.mcool")
tss = Tilesets().add(ts)
track = ts.track("heatmap")
hg.view(track, width=6).widget(ts=tss)